# ТУТ 10 СЕКУНД ОКАЗЫВАЕТСЯ

In [31]:
import os
import csv
import wfdb
import heartpy as hp
import pandas as pd
import numpy as np
from scipy.signal import resample, butter, filtfilt

In [32]:
def funcBPFilter(ppg_signal, order, f1, f2, fs):
    fc = [f1, f2]
    b, a = butter(order, fc, btype='bandpass', analog=False, output='ba', fs=fs)
    signalBP = filtfilt(b, a, ppg_signal)
    return signalBP

In [33]:
age_group = {15:1.0,16:1.0,17:1.0,18:1.0,19:1.0,
             20:2.0,21:2.0,22:2.0,23:2.0,24:2.0,
             25:3.0,26:3.0,27:3.0,28:3.0,29:3.0,
             30:4.0,31:4.0,32:4.0,33:4.0,34:4.0,
             35:5.0,36:5.0,37:5.0,38:5.0,39:5.0,
             40:6.0,41:6.0,42:6.0,43:6.0,44:6.0,
             45:7.0,46:7.0,47:7.0,48:7.0,49:7.0,
             50:8.0,51:8.0,52:8.0,53:8.0,54:8.0,
             55:9.0,56:9.0,57:9.0,58:9.0,59:9.0,
             60:10.0,61:10.0,62:10.0,63:10.0,64:10.0,
             65:11.0,66:11.0,67:11.0,68:11.0,69:11.0,
             70:12.0,71:12.0,72:12.0,73:12.0,74:12.0,
             75:13.0,76:13.0,77:13.0,78:13.0,79:13.0,
             80:14.0,81:14.0,82:14.0,83:14.0,84:14.0,
             85:15.0,86:15.0,87:15.0,88:15.0,89:15.0,
             90:16.0,91:16.0,92:16.0,93:16.0,94:16.0,
             95:17.0,96:17.0,97:17.0,98:17.0,99:17.0,100:17.0,
             }

In [34]:
subjects = pd.read_csv(r"D:\Proga\AML\datasets\bruno\subject-info.csv",dtype = {"ID":str})
subjects.dropna(subset=["Age [years]"],inplace=True)
subjects = subjects.reset_index()
subjects = subjects[["ID","Age [years]"]]
subjects["Age [years]"] = subjects["Age [years]"].map(age_group)
subjects["Age [years]"]

0       8.0
1       8.0
2       8.0
3       8.0
4       8.0
       ... 
3883    8.0
3884    8.0
3885    8.0
3886    8.0
3887    8.0
Name: Age [years], Length: 3888, dtype: float64

In [35]:
ids = subjects["ID"].values
age_groups = subjects["Age [years]"].values

In [36]:
result = []
result_metadata = []
result_signals = {}

target_fs = 1000
five_minute_samples = 300 * target_fs
new_id = 1147

for id in ids:
    new_id += 1 
    try:
        record = wfdb.rdrecord('D:\\Proga\\AML\\datasets\\bruno\\' + id + "\\"+ id+"_PPG")

        #print(record.__dict__)
        #print(record.p_signal.shape)
        #channel_index = record.sig_name.index('NIBP')
        signal = record.p_signal.astype(np.float32).flatten()
        fs = record.fs

        signal = signal[:five_minute_samples]

        if fs != target_fs:
            num_samples = int(len(signal) * target_fs / fs)
            signal = resample(signal, num_samples).astype(np.float32).flatten()
            fs = target_fs

        signal = signal[:five_minute_samples]
        
        try:
            wd, m = hp.process(signal, fs)
            idx = int(subjects[subjects["ID"] == id].index.to_list()[0])
            age_group = age_groups[idx]
            
            result_metadata.append({
                'id': new_id,
                'age_group': age_group,
                **m
            })
            
            result_signals[new_id] = signal
            
            print((new_id, id, age_group, signal, m))

        except hp.exceptions.BadSignalWarning as e:
            result_metadata.append({
                'id': new_id,
                'age_group': None,
                'error': "BadSignalWarning"
            })
            print((new_id, id, None, None, None, "BadSignalWarning"))

    except FileNotFoundError as e:
        result_metadata.append({
            'id': new_id,
            'age_group': None,
            'error': "FileNotFound"
        })
        print((new_id, id, None, None, None, "FileNotFound"))

    print(f"Process end: {id}")

(1148, '100001', 8.0, array([-254.17479, -254.18237, -254.19066, ..., -254.15694, -254.16206,
       -254.16803], dtype=float32), {'bpm': 159.76331360946747, 'ibi': 375.55555555555554, 'sdnn': 133.7876519058061, 'sdsd': 90.11967352113275, 'rmssd': 249.1993846969397, 'pnn20': 1.0, 'pnn50': 1.0, 'hr_mad': 128.0, 'sd1': 167.35640611182671, 'sd2': 35.293216219425375, 's': 18555.96097697151, 'sd1/sd2': 4.741885949734266, 'breathingrate': 0.2958579881656805})
Process end: 100001
(1149, '100002', 8.0, array([-253.86581, -253.86165, -253.85788, ..., -253.88097, -253.87556,
       -253.87048], dtype=float32), {'bpm': 199.11504424778764, 'ibi': 301.3333333333333, 'sdnn': 177.04770229768272, 'sdsd': 159.85835567010884, 'rmssd': 281.8598942737331, 'pnn20': 0.8571428571428571, 'pnn50': 0.7619047619047619, 'hr_mad': 174.5, 'sd1': 199.28988446453224, 'sd2': 149.30897517573152, 's': 93480.50344596713, 'sd1/sd2': 1.3347481906561538, 'breathingrate': 0.27654867256637167})
Process end: 100002
(1150, '100

c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\heartpy\analysis.py:677: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  interp = UnivariateSpline(x, rrlist, k=3)


(1169, '105002', 2.0, array([-253.46623, -253.47223, -253.47784, ..., -253.4462 , -253.45314,
       -253.45995], dtype=float32), {'bpm': 113.59333585762968, 'ibi': 528.2, 'sdnn': 163.46995646499286, 'sdsd': 133.73540953432493, 'rmssd': 189.75368649504952, 'pnn20': 0.6923076923076923, 'pnn50': 0.6153846153846154, 'hr_mad': 58.0, 'sd1': 134.1189527662823, 'sd2': 199.3712305573813, 's': 84004.49315181418, 'sd1/sd2': 0.6727096602219212, 'breathingrate': 0.3786444528587656})
Process end: 105002
(1170, '105003', 2.0, array([-253.79192, -253.791  , -253.79019, ..., -253.79546, -253.79419,
       -253.79295], dtype=float32), {'bpm': 160.9538002980626, 'ibi': 372.77777777777777, 'sdnn': 194.5734809941294, 'sdsd': 155.01744470628233, 'rmssd': 340.2247156554872, 'pnn20': 0.9285714285714286, 'pnn50': 0.9285714285714286, 'hr_mad': 166.0, 'sd1': 240.56833218231276, 'sd2': 153.69483413835457, 's': 116157.59207790505, 'sd1/sd2': 1.5652336887637714, 'breathingrate': 0.29806259314456035})
Process end: 

c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\ma\core.py:5317: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3747: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


(1180, '108001', 2.0, array([-137.47482, -137.48573, -137.49562, ..., -137.4362 , -137.45004,
       -137.46295], dtype=float32), {'bpm': 116.991643454039, 'ibi': 512.8571428571429, 'sdnn': 179.74301382921163, 'sdsd': 102.25305618904503, 'rmssd': 214.66776656032923, 'pnn20': 1.0, 'pnn50': 1.0, 'hr_mad': 212.0, 'sd1': 148.45822223777301, 'sd2': 215.60923855438105, 's': 100559.12694882964, 'sd1/sd2': 0.6885522310322005, 'breathingrate': 0.2785515320334262})
Process end: 108001
(1181, '108002', 2.0, array([-137.16336, -137.19566, -137.22588, ..., -137.05472, -137.09283,
       -137.12904], dtype=float32), {'bpm': 148.0872069107363, 'ibi': 405.1666666666667, 'sdnn': 136.08014386954315, 'sdsd': 88.88862413155015, 'rmssd': 243.55030281237592, 'pnn20': 1.0, 'pnn50': 1.0, 'hr_mad': 126.5, 'sd1': 167.64350196175215, 'sd2': 53.327701525567385, 's': 28085.972265892015, 'sd1/sd2': 3.14364761964056, 'breathingrate': 0.8227067050596463})
Process end: 108002
(1182, '108003', 2.0, array([-136.73552, -

In [37]:
df_metadata = pd.DataFrame(result_metadata)
#df_metadata.dropna(inplace=True)
df_metadata

,id,age_group,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate,error
0,1148,8.0,159.763314,375.555556,133.787652,90.119674,249.199385,1.000000,1.000000,128.0,167.356406,35.293216,18555.960977,4.741886,0.295858,NaN
1,1149,8.0,199.115044,301.333333,177.047702,159.858356,281.859894,0.857143,0.761905,174.5,199.289884,149.308975,93480.503446,1.334748,0.276549,NaN
2,1150,8.0,115.690528,518.625000,225.265919,185.994193,274.292545,0.800000,0.600000,167.0,193.792363,226.314560,137784.075404,0.856296,0.241022,NaN
3,1151,8.0,149.897797,400.272727,141.548637,162.823092,213.517856,0.750000,0.750000,59.0,144.065713,89.406332,40464.929057,1.611359,0.227118,NaN
4,1152,8.0,66.239788,905.800000,51.287035,58.903143,91.061030,0.666667,0.444444,31.0,64.385989,34.336749,6945.450474,1.875134,0.331199,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3883,5031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BadSignalWarning
3884,5032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BadSignalWarning
3885,5033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BadSignalWarning
3886,5034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BadSignalWarning


In [38]:
df_metadata.to_csv(r"D:\Proga\AML\PPG_dataset\dataset.csv",mode="a",index=False,header=False)

In [ ]:
df = pd.read_csv(r"D:\Proga\AML\PPG_dataset\dataset.csv")
df.dropna(inplace=True)